In [23]:
# load libraries

from bs4 import BeautifulSoup
import requests
import smtplib
import time
import datetime
import csv

    

In [27]:
# Connect to website (this should be static page)

URL = 'https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ/ref=sr_1_9?crid=198UHNX324P4A&keywords=data+analyst+tshirt&qid=1666878135&qu=eyJxc2MiOiI0LjgzIiwicXNhIjoiMy45NyIsInFzcCI6IjIuOTMifQ%3D%3D&sprefix=data+analyst+tshirt%2Caps%2C100&sr=8-9'

# get header information for computer, type httpbin.org/get into web browser and a couple other file readers for java/json
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

#start pulling data

page = requests.get(URL, headers = headers)

soup1 = BeautifulSoup(page.content, "html.parser")

# look at initial data pull (html format, inspect webpage to see this code)
# print(soup1)

# format the html, make it 'pretty'

soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

# print(soup2)

# now get data of interest, go to webpage and inpect code, right click inpsect, or ctrl + shift + i)
# note, use the strip() function to get rid of white space

title = soup2.find(id='productTitle').get_text().strip()

# print(title)

# finding price on Amazon has changed, it used to be id="priceblock_ourprice" but October 2022 
# is now class="a-offscreen" and because class is a funcion (?) in python need to have code as 
# class_ (underscore after class)
# use to strip to remoe whitespace and to remove $ sign, [1:] will take everything from 1st position onward

price = soup2.find(class_ = 'a-offscreen').get_text()
price = price.strip()[1:]

#price = soup2.xpath('//span[@class="a-offscreen"]/text()')[0]
#price = price.replace(',', '').replace('₹', '').replace('.00', '')

print(title)
print(price)

# put a time stamp on when data was collected

today = datetime.date.today()

print(today)




Funny Got Data MIS Data Systems Business Analyst T-Shirt
16.99
2022-10-27


In [28]:
# make CSV file to put data into

header = ['ProductTitle', 'Price', 'CollectionDate']
data = [title, price, today]

# check datatype, csv is a list (price is a string)
type(data)

# 'w' means write  
# comment these next 4 lines out if you want to restart (say after computer restart) the script if you have running in background
with open('AmazonWebScraperDataset.csv', 'w', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(header) # intitial insertion of data into csv
    writer.writerow(data) # now insert data

In [29]:
# use Pandas to check data was inserted into csv file correctly

import pandas as pd

df = pd.read_csv(r'C:\Users\benja\AmazonWebScraperDataset.csv')

print(df)

                                        ProductTitle  Price CollectionDate
0  Funny Got Data MIS Data Systems Business Analy...  16.99     2022-10-27


In [ ]:
# append data to the csv file (price tracker over time); 'a+' appends the csv/dataset

with open('AmazonWebScraperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data) # now insert data

In [30]:
# automate

def check_price():
    URL = 'https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ/ref=sr_1_9?crid=198UHNX324P4A&keywords=data+analyst+tshirt&qid=1666878135&qu=eyJxc2MiOiI0LjgzIiwicXNhIjoiMy45NyIsInFzcCI6IjIuOTMifQ%3D%3D&sprefix=data+analyst+tshirt%2Caps%2C100&sr=8-9'

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    page = requests.get(URL, headers = headers)

    soup1 = BeautifulSoup(page.content, "html.parser")

    soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

    title = soup2.find(id='productTitle').get_text().strip()

    price = soup2.find(class_ = 'a-offscreen').get_text()
    price = price.strip()[1:]
    
    today = datetime.date.today()
    
    header = ['ProductTitle', 'Price', 'CollectionDate']
    data = [title, price, today]
    
    with open('AmazonWebScraperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(data)
        
    if(price < 15):
        send_mail()
    
    
    

In [31]:
while(True):
    check_price()
    time.sleep(86400) # set time in seconds, initially try 5 seconds to test code

KeyboardInterrupt: 

In [32]:
df = pd.read_csv(r'C:\Users\benja\AmazonWebScraperDataset.csv')

print(df)

                                        ProductTitle  Price CollectionDate
0  Funny Got Data MIS Data Systems Business Analy...  16.99     2022-10-27
1  Funny Got Data MIS Data Systems Business Analy...  16.99     2022-10-28
2  Funny Got Data MIS Data Systems Business Analy...  16.99     2022-10-28


In [ ]:
# send email if price falls below a certain level

def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    #server.starttls()
    server.ehlo()
    server.login('my_email@gmail.com','email_password')
    
    subject = "The Shirt you want is below $15! Now is your chance to buy!"
    body = "Finally, a price drop. Now is your chance to pick up the shirt of your dreams. Act now! Link here: https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ/ref=sr_1_3?dchild=1&keywords=data+analyst+tshirt&qid=1626655184&sr=8-3"
   
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(
        'my_email@gmail.com',
        msg
     
    )